In [1]:
import MetaTrader5 as mt5
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
import mplfinance as mpf
from mplfinance import plot as mplot
import os
from tensorflow import keras
from matplotlib.image import imread
from datetime import date
import datetime
from datetime import datetime as dt
import matplotlib 
matplotlib.use('agg')
import matplotlib.pyplot as plt
import re
import tensorflow
import cv2 as cv
from tqdm import tqdm
from PIL import ImageFont, ImageDraw, Image
import gc
from datetime import datetime
import pytz
import warnings
warnings.filterwarnings('ignore')

In [4]:
mt5.copy_rates_from_pos("EURUSD", mt5.TIMEFRAME_H1, 0, 50)

array([(1642068000, 1.14642, 1.14775, 1.14615, 1.14655, 3243, 14, 0),
       (1642071600, 1.14657, 1.14742, 1.14577, 1.14706, 2441, 14, 0),
       (1642075200, 1.14706, 1.14768, 1.14567, 1.14599, 2443, 14, 0),
       (1642078800, 1.14599, 1.14689, 1.14578, 1.14618, 2142, 13, 0),
       (1642082400, 1.1462 , 1.14692, 1.14525, 1.14578, 2701, 14, 0),
       (1642086000, 1.14579, 1.14711, 1.14571, 1.14711, 3869, 14, 0),
       (1642089600, 1.14713, 1.1481 , 1.14674, 1.1473 , 3595, 14, 0),
       (1642093200, 1.1473 , 1.14768, 1.14464, 1.14597, 4635, 14, 0),
       (1642096800, 1.14599, 1.14712, 1.1456 , 1.14645, 2944, 14, 0),
       (1642100400, 1.14645, 1.14665, 1.14543, 1.14605, 2058, 14, 0),
       (1642104000, 1.14605, 1.14656, 1.1457 , 1.14578, 1282, 14, 0),
       (1642107600, 1.14579, 1.14645, 1.14569, 1.1461 , 1276, 13, 0),
       (1642111200, 1.14609, 1.14616, 1.14502, 1.14514, 1510, 14, 0),
       (1642114800, 1.14512, 1.14549, 1.14503, 1.1454 ,  505, 13, 0),
       (1642118400, 

In [2]:
def get_position_sell(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_SELL,
        "price": price,
        "sl": price + (sl * price),
        "tp": price - (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [3]:
def get_position_buy(symbol, lot, sl , tp):
    symbol_info = mt5.symbol_info(symbol)
    point = mt5.symbol_info(symbol).point
    price = mt5.symbol_info_tick(symbol).bid
    deviation = 20
    print(price + (sl * point))
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": mt5.ORDER_TYPE_BUY,
        "price": price,
        "sl": price - (sl * price),
        "tp": price + (tp * price),
        "deviation": deviation,
        "comment": "python script open",
        "type_filling":mt5.SYMBOL_TRADE_EXECUTION_INSTANT,
    }
    result = mt5.order_send(request)
    return result

In [4]:
currency_pairs= ['AUDCADb', 'AUDCHFb','AUDJPYb','AUDNZDb','AUDUSDb','CADCHFb','CADJPYb','CHFJPYb',
                 'EURAUDb','EURCADb','EURCHFb','EURGBPb','EURJPYb','EURNZDb','EURUSDb',
                 'GBPAUDb','GBPCADb','GBPCHFb','GBPJPYb','GBPNZDb','GBPUSDb','NZDJPYb',
                 'NZDCADb','NZDCHFb','NZDUSDb','USDCADb','USDCHFb','USDJPYb']

In [2]:
# connect to MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

mt5.account_info()

AccountInfo(login=5386640, trade_mode=0, leverage=500, limit_orders=0, margin_so_mode=0, trade_allowed=True, trade_expert=True, margin_mode=2, currency_digits=2, fifo_close=False, balance=10224.66, credit=0.0, profit=-304.82, equity=9919.84, margin=128.12, margin_free=9791.72, margin_level=7742.616297221355, margin_so_call=50.0, margin_so_so=20.0, margin_initial=0.0, margin_maintenance=0.0, assets=0.0, liabilities=0.0, commission_blocked=0.0, name='Zahed Zahedi', server='AMarkets-Demo', currency='USD', company='AMarkets')

In [6]:
image_path_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h'
image_path_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h'
RecolPath_1h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_1h\Recolored'
RecolPath_4h = r'C:\Users\Administrator\Desktop\Trade_1005\Images_4h\Recolored'
model_1h= tensorflow.keras.models.load_model('ckpt_03_CorrectHighLowWithoutResize.hdf5')
model_4h_old= tensorflow.keras.models.load_model('ckpt_04_4h_best.hdf5')
model_4h_new= tensorflow.keras.models.load_model('ckpt_04_tp008sl003_best_4h.hdf5')
df_minmax_1h= pd.read_csv('df_minmax_mrged_1h.csv')
df_minmax_4h= pd.read_csv('df_minmax_merged_4h.csv')
tesh_optimum_1h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_CorrectHL.csv')
tesh_optimum_4h = pd.read_csv('df_best_final_EmbedRecolor_withoutResize_4h.csv')
# Selected_pairs_for_Old_model_4h= ['CHFJPYb', 'AUDUSDb', 'GBPCADb', 'EURJPYb', 'GBPJPYb', 'EURNZDb', 'EURGBPb',
#                                   'NZDUSDb', 'AUDCHFb', 'AUDCADb', 'NZDCHFb', 'EURCADb', 'EURUSDb', 'GBPUSDb', 'NZDCADb']
# Selected_pairs_for_New_model_4h= ['USDCADb', 'EURAUDb', 'EURCHFb', 'GBPAUDb', 'CADJPYb', 'AUDNZDb',
#                                   'USDJPYb', 'CADCHFb', 'GBPCHFb', 'AUDJPYb', 'GBPNZDb', 'NZDJPYb', 'USDCHFb']

In [7]:
Selected_pairs_for_Old_model_4h= ['CHFJPYb', 'AUDUSDb', 'GBPCADb', 'EURJPYb', 'GBPJPYb', 'EURNZDb', 'EURGBPb',
                                  'NZDUSDb', 'AUDCHFb', 'AUDCADb', 'NZDCHFb', 'EURCADb', 'EURUSDb', 'GBPUSDb', 'NZDCADb']
Selected_pairs_for_New_model_4h= ['USDCADb', 'EURAUDb', 'EURCHFb', 'GBPAUDb', 'CADJPYb', 'AUDNZDb',
                                  'USDJPYb', 'CADCHFb', 'GBPCHFb', 'AUDJPYb', 'GBPNZDb', 'NZDJPYb', 'USDCHFb']

In [8]:
mc = mpf.make_marketcolors(up='g',down='r')
s  = mpf.make_mpf_style(marketcolors=mc)

In [9]:
def make_candle_stick(df, image_path, pair):
    df['time']= df.time.apply(lambda x: dt.utcfromtimestamp(x).strftime('%Y%m%d %H%M%S'))
    df['time']= pd.to_datetime(df['time'])
    time_log= df.iloc[48]['time']
    
    openn = df.iloc[48]['open']
    high = df.iloc[48]['high']
    low = df.iloc[48]['low']
    close = df.iloc[48]['close']
    df = df.set_index('time')
    df.rename(columns={'tick_volume':'volume'}, inplace=True)
    
    HighLow= str(np.max(df.iloc[1:, 1])/np.min(df.iloc[1:, 2])).replace('.', '')[:6]
    if '_' in HighLow:
        HighLow = HighLow.replace('_', '0')
    
    #Heiken Ashi
    df_temp_Ha = df.copy()
    for i in range(1, len(df_temp_Ha)):
        df_temp_Ha.iloc[i, 0] = (df_temp_Ha.iloc[i-1, 0] + df_temp_Ha.iloc[i-1, 3])/2
        df_temp_Ha.iloc[i, 3] = (df_temp_Ha.iloc[i, 0] + df_temp_Ha.iloc[i, 1] + df_temp_Ha.iloc[i, 2] + df_temp_Ha.iloc[i, 3])/4
    
    time_log = str(time_log).replace(':', '').replace(' ', '').replace('-', '')
    
    img_name = "".join([pair, '_' ,str(time_log),'_', HighLow, '.jpg'])
    img_path= os.path.join(image_path, img_name)
    
    mplot(df_temp_Ha.iloc[1:-1, :], type='candle', volume=False, figscale=2,figratio=(10,10), style=s, savefig=img_path)
    return time_log, img_path, openn, high, low, close


In [10]:
# font = ImageFont.truetype('times.ttf', 20)
font = cv.FONT_HERSHEY_COMPLEX
loc= (600, 795)
font_scale= 1.1
color = (0, 0, 0)
thickness= 3

# print(df_minmax)

In [11]:
def makeEmbedImage(img_name, RecolPath, df_minmax, pair):
    img = cv.imread(img_name)
    img = img[140:-207, 220: -120, :]
    hl = int(img_name.split('.')[0].split('_')[-1])
    if hl < 100:
        hl = hl * 10000
    elif hl < 1000:
        hl = hl * 1000
    elif hl < 10000:
        hl = hl * 100
    elif hl < 100000:
        hl = hl * 10

    mx = df_minmax.loc[0, pair]
    mn = df_minmax.loc[1, pair]
    hl = (hl - mn) / (mx - mn)
    w = np.uint8(((hl * 255)))
    img[771:850, 20:550, 2] = w
    img[771:850, 20:550, 0] = w
    img[771:850, 20:550, 1] = 0
    img = cv.putText(img, pair, loc, font, font_scale, color, thickness, cv.LINE_AA)
    img = cv.resize(img, (300, 300))
    name= img_name.split('\\')[-1]
    cv.imwrite(os.path.join(RecolPath, name), img)
    return img


In [12]:
list_log = []
def make_predict(i):
        list_log_dashboard= []
        for pair in currency_pairs:
#             tp= tesh_optimum.loc[tesh_optimum['p']== pair,'t'].values[0]
#             sl= tesh_optimum.loc[tesh_optimum['p']== pair,'s'].values[0]
#             acc_u= tesh_optimum.loc[tesh_optimum['p']== pair,'u'].values[0]
#             acc_d= tesh_optimum.loc[tesh_optimum['p']== pair,'d'].values[0]
#             tp= 0.003
#             sl= 0.004
#             acc_u= 0.99
#             acc_d= 0.01
            
            rates_1h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H1, 0, 50)
            df_1h=pd.DataFrame(rates_1h).reset_index(drop=True)
            time_log, img_name_1h, open_1h, high_1h, low_1h, close_1h = make_candle_stick(df_1h, image_path_1h, pair[:-1])
            img_1h= makeEmbedImage(img_name_1h, RecolPath_1h, df_minmax_1h, pair[:-1])
            acc_1h=model_1h.predict(np.array([img_1h,img_1h]))[0,1]
            
            rates_4h = mt5.copy_rates_from_pos(pair, mt5.TIMEFRAME_H4, 0, 50)
            df_4h=pd.DataFrame(rates_4h).reset_index(drop=True)
            time_log, img_name_4h, open_4h, high_4h, low_4h, close_4h = make_candle_stick(df_4h, image_path_4h, pair[:-1])
            img_4h= makeEmbedImage(img_name_4h, RecolPath_4h, df_minmax_4h, pair[:-1])
            if pair in Selected_pairs_for_Old_model_4h:
                acc_4h=model_4h_old.predict(np.array([img_4h,img_4h]))[0,1]
            else:
                acc_4h=model_4h_new.predict(np.array([img_4h,img_4h]))[0,1]
            print(f'{pair}, acc_1h:{acc_1h}, acc_4h:{acc_4h}')
            
#             if acc_1h >= acc_u:
#                 print('------ Buy Position -------')
#                 get_position_buy(pair, 0.1,sl , tp)
#             elif acc_1h <= acc_d:
#                 print('--------- Sell Position --------')
#                 get_position_sell(pair, 0.1,sl , tp)
            
            list_log.append([time_log, pair, img_name_1h, img_name_4h, acc_1h, acc_4h,open_1h, high_1h, low_1h, close_1h, open_4h, high_4h, low_4h, close_4h])
            list_log_dashboard.append([pair, acc_1h, acc_4h])
            
            if pair == currency_pairs[-1]:
                arr_log_dashboard = np.array(list_log_dashboard)
                arr_log_dashboard= arr_log_dashboard.reshape(int(len(currency_pairs)), 3)
                df_dashboard= pd.DataFrame(arr_log_dashboard, columns=['PAIR','1H', '4H'])
                df_dashboard.to_excel(r'C:\xampp\htdocs\fx_dashboard\pred.xlsx', index= False)
                
                
                
                arr_log = np.array(list_log)
                arr_log= arr_log.reshape(int(len(currency_pairs)*i), 14)
                df_temp = pd.DataFrame(arr_log, columns=['time_log','pair', 'img_name_1h','img_name_4h', 'acc_1h',
                                                         'acc_4h','open_1h', 'high_1h',  'low_1h','close_1h',
                                                        'open_4h','high_4h', 'low_4h', 'close_4h'])
                if i == 1:
                    df_log = df_temp
                else:
                    df_log = pd.concat([df_log, df_temp], axis = 0)
                    
                file_name= r'C:\Users\Administrator\Desktop\Trade_1005\Logs_1005/df_log_{0}_4.csv'.format(str(i))
                df_log.to_csv(file_name)

In [13]:
i = 0
while(True):
    time = dt.now()
    min_part= str(time).split(':')[1]
    sec_part= str(time).split(':')[-1].split('.')[0]
    if (min_part == '59') & (int(sec_part) >= 48):
        print(f'---- {time} ------ ')
        i+= 1
        try:
            make_predict(i)
        except:
            print(f'----!!!! except !!!! ------ ')
            

---- 2022-01-11 00:59:48.004511 ------ 
AUDCADb, acc_1h:0.3870297372341156, acc_4h:0.11691933870315552
AUDCHFb, acc_1h:0.8214887976646423, acc_4h:0.24102181196212769
AUDJPYb, acc_1h:0.0818159207701683, acc_4h:0.019976496696472168
AUDNZDb, acc_1h:0.5180403590202332, acc_4h:0.32702159881591797
AUDUSDb, acc_1h:0.6877917647361755, acc_4h:0.13656572997570038
CADCHFb, acc_1h:0.6759555339813232, acc_4h:0.756187915802002
CADJPYb, acc_1h:0.44074344635009766, acc_4h:0.4577048420906067
CHFJPYb, acc_1h:0.0021224389784038067, acc_4h:0.1820448786020279
EURAUDb, acc_1h:0.49785876274108887, acc_4h:0.44295868277549744
EURCADb, acc_1h:0.05085940286517143, acc_4h:0.08614283800125122
EURCHFb, acc_1h:0.7105892300605774, acc_4h:0.4538412094116211
EURGBPb, acc_1h:0.20415514707565308, acc_4h:0.2597263753414154
EURJPYb, acc_1h:0.009045680984854698, acc_4h:0.17091462016105652
EURNZDb, acc_1h:0.5491554141044617, acc_4h:0.2029964029788971
EURUSDb, acc_1h:0.6549006700515747, acc_4h:0.5219408869743347
GBPAUDb, acc_

EURUSDb, acc_1h:0.025063371285796165, acc_4h:0.23075063526630402
GBPAUDb, acc_1h:0.19572556018829346, acc_4h:0.39217010140419006
GBPCADb, acc_1h:0.4932999610900879, acc_4h:0.07250786572694778
GBPCHFb, acc_1h:0.5228338837623596, acc_4h:0.9972768425941467
GBPJPYb, acc_1h:0.001537399017252028, acc_4h:0.016908586025238037
GBPNZDb, acc_1h:0.3085581958293915, acc_4h:0.7668720483779907
GBPUSDb, acc_1h:0.006614352110773325, acc_4h:0.09426288306713104
NZDJPYb, acc_1h:0.0010037344181910157, acc_4h:0.21332061290740967
NZDCADb, acc_1h:0.3042244017124176, acc_4h:0.28025779128074646
NZDCHFb, acc_1h:0.4443798065185547, acc_4h:0.5373099446296692
NZDUSDb, acc_1h:0.020370513200759888, acc_4h:0.04948831722140312
USDCADb, acc_1h:0.5493326783180237, acc_4h:0.6093987822532654
USDCHFb, acc_1h:0.615606427192688, acc_4h:0.9991462230682373
USDJPYb, acc_1h:0.0021579640451818705, acc_4h:0.8722696900367737
----!!!! except !!!! ------ 
---- 2022-01-11 05:59:48.008991 ------ 
AUDCADb, acc_1h:0.34881719946861267, acc

USDJPYb, acc_1h:0.11670487374067307, acc_4h:0.2660427689552307
----!!!! except !!!! ------ 
---- 2022-01-11 09:59:48.006856 ------ 
AUDCADb, acc_1h:0.22639314830303192, acc_4h:0.21132372319698334
AUDCHFb, acc_1h:0.2933156490325928, acc_4h:0.08257339894771576
AUDJPYb, acc_1h:0.1582670360803604, acc_4h:0.06855233013629913
AUDNZDb, acc_1h:0.3417544960975647, acc_4h:0.3119574785232544
AUDUSDb, acc_1h:0.32704854011535645, acc_4h:0.20042340457439423
CADCHFb, acc_1h:0.875133752822876, acc_4h:0.25603142380714417
CADJPYb, acc_1h:0.053075384348630905, acc_4h:0.007387165445834398
CHFJPYb, acc_1h:0.08985520154237747, acc_4h:0.0809667631983757
EURAUDb, acc_1h:0.42151305079460144, acc_4h:0.9252471327781677
EURCADb, acc_1h:0.019836243242025375, acc_4h:0.04058355838060379
EURCHFb, acc_1h:0.3556240499019623, acc_4h:0.9995352029800415
EURGBPb, acc_1h:0.044981490820646286, acc_4h:0.01918857917189598
EURJPYb, acc_1h:0.16903117299079895, acc_4h:0.20659834146499634
EURNZDb, acc_1h:0.11617012321949005, acc_4

EURJPYb, acc_1h:0.24328108131885529, acc_4h:0.47403445839881897
EURNZDb, acc_1h:0.0020827148109674454, acc_4h:0.1903442144393921
EURUSDb, acc_1h:0.19276264309883118, acc_4h:0.1122930571436882
GBPAUDb, acc_1h:0.16532401740550995, acc_4h:0.5368291139602661
GBPCADb, acc_1h:0.21421286463737488, acc_4h:0.39910465478897095
GBPCHFb, acc_1h:0.33097562193870544, acc_4h:0.9083919525146484
GBPJPYb, acc_1h:0.2020304650068283, acc_4h:0.4134913384914398
GBPNZDb, acc_1h:0.00039941523573361337, acc_4h:0.3085215985774994
GBPUSDb, acc_1h:0.27979084849357605, acc_4h:0.03747999295592308
NZDJPYb, acc_1h:0.2762654423713684, acc_4h:0.6793015599250793
NZDCADb, acc_1h:0.5305341482162476, acc_4h:0.2835724949836731
NZDCHFb, acc_1h:0.32603898644447327, acc_4h:0.8194194436073303
NZDUSDb, acc_1h:0.595460832118988, acc_4h:0.023235315456986427
USDCADb, acc_1h:0.34608906507492065, acc_4h:0.1799393892288208
USDCHFb, acc_1h:0.2958623468875885, acc_4h:0.3671543002128601
USDJPYb, acc_1h:0.1897641271352768, acc_4h:0.389429

USDCHFb, acc_1h:0.4566923677921295, acc_4h:0.2380673885345459
USDJPYb, acc_1h:0.16926319897174835, acc_4h:0.7349719405174255
----!!!! except !!!! ------ 
---- 2022-01-11 18:59:48.007921 ------ 
AUDCADb, acc_1h:0.3043096363544464, acc_4h:0.2774715721607208
AUDCHFb, acc_1h:0.20212891697883606, acc_4h:0.5971031785011292
AUDJPYb, acc_1h:0.08536294847726822, acc_4h:0.01922021061182022
AUDNZDb, acc_1h:0.26838985085487366, acc_4h:0.26528871059417725
AUDUSDb, acc_1h:0.17219461500644684, acc_4h:0.1033453568816185
CADCHFb, acc_1h:0.4904269278049469, acc_4h:0.574206531047821
CADJPYb, acc_1h:0.13388574123382568, acc_4h:0.1240835040807724
CHFJPYb, acc_1h:0.08804532885551453, acc_4h:0.2966136634349823
EURAUDb, acc_1h:0.46162211894989014, acc_4h:0.07210718840360641
EURCADb, acc_1h:0.4828055799007416, acc_4h:0.22466248273849487
EURCHFb, acc_1h:0.3580683767795563, acc_4h:0.5836824178695679
EURGBPb, acc_1h:0.3046521544456482, acc_4h:0.5774884223937988
EURJPYb, acc_1h:0.06985016912221909, acc_4h:0.097195

...

GBPCHFb, acc_1h:0.32341066002845764, acc_4h:0.008979354985058308
GBPJPYb, acc_1h:0.32844099402427673, acc_4h:0.2877679765224457
GBPNZDb, acc_1h:0.5427085161209106, acc_4h:0.7929561138153076
GBPUSDb, acc_1h:0.02470075525343418, acc_4h:0.07389049232006073
NZDJPYb, acc_1h:0.4077141284942627, acc_4h:0.3991956114768982
NZDCADb, acc_1h:0.16743069887161255, acc_4h:0.040794242173433304
NZDCHFb, acc_1h:0.337797075510025, acc_4h:0.07899034768342972
NZDUSDb, acc_1h:0.28199902176856995, acc_4h:0.009904037229716778
USDCADb, acc_1h:0.268846333026886, acc_4h:0.3808317184448242
USDCHFb, acc_1h:0.37190601229667664, acc_4h:0.19886766374111176
USDJPYb, acc_1h:0.5561548471450806, acc_4h:0.9047454595565796
----!!!! except !!!! ------ 
---- 2022-01-12 04:59:48.010716 ------ 
AUDCADb, acc_1h:0.031266916543245316, acc_4h:0.059736672788858414
AUDCHFb, acc_1h:0.2839602828025818, acc_4h:0.20880918204784393
AUDJPYb, acc_1h:0.27840062975883484, acc_4h:0.324131578207016
AUDNZDb, acc_1h:0.2169947624206543, acc_4h:0.

AUDJPYb, acc_1h:0.4374938905239105, acc_4h:0.6222416758537292
AUDNZDb, acc_1h:0.3134598433971405, acc_4h:0.7835846543312073
AUDUSDb, acc_1h:0.19117659330368042, acc_4h:0.28300315141677856
CADCHFb, acc_1h:0.9469171166419983, acc_4h:0.8181236386299133
CADJPYb, acc_1h:0.6859565377235413, acc_4h:0.9576747417449951
CHFJPYb, acc_1h:0.19729794561862946, acc_4h:0.37916308641433716
EURAUDb, acc_1h:0.288183331489563, acc_4h:0.12541785836219788
EURCADb, acc_1h:0.2730187177658081, acc_4h:0.2720431983470917
EURCHFb, acc_1h:0.5310280919075012, acc_4h:0.36896631121635437
EURGBPb, acc_1h:0.4619024693965912, acc_4h:0.30529022216796875
EURJPYb, acc_1h:0.31384196877479553, acc_4h:0.6275606155395508
EURNZDb, acc_1h:0.23045285046100616, acc_4h:0.47845375537872314
EURUSDb, acc_1h:0.08766400814056396, acc_4h:0.9907224178314209
GBPAUDb, acc_1h:0.11369835585355759, acc_4h:0.42520666122436523
GBPCADb, acc_1h:0.1127019003033638, acc_4h:0.1452704221010208
GBPCHFb, acc_1h:0.45479676127433777, acc_4h:0.216655507683

GBPCADb, acc_1h:0.4132499396800995, acc_4h:0.2734745144844055
GBPCHFb, acc_1h:0.5478019714355469, acc_4h:0.5285142660140991
GBPJPYb, acc_1h:0.2872998118400574, acc_4h:0.3182432949542999
GBPNZDb, acc_1h:0.23403224349021912, acc_4h:0.4957800507545471
GBPUSDb, acc_1h:0.34475523233413696, acc_4h:0.5954796075820923
NZDJPYb, acc_1h:0.3569232225418091, acc_4h:0.4343935251235962
NZDCADb, acc_1h:0.359588623046875, acc_4h:0.13473151624202728
NZDCHFb, acc_1h:0.4679222106933594, acc_4h:0.06232236698269844
NZDUSDb, acc_1h:0.1809566169977188, acc_4h:0.4986252784729004
USDCADb, acc_1h:0.10341405868530273, acc_4h:0.4322957694530487
USDCHFb, acc_1h:0.3130201995372772, acc_4h:0.22879812121391296
USDJPYb, acc_1h:0.2612760663032532, acc_4h:0.5512102246284485
----!!!! except !!!! ------ 
---- 2022-01-12 13:59:48.012296 ------ 
AUDCADb, acc_1h:0.19917190074920654, acc_4h:0.2266346514225006
AUDCHFb, acc_1h:0.3409426510334015, acc_4h:0.534245491027832
AUDJPYb, acc_1h:0.18261227011680603, acc_4h:0.283712446689

AUDCHFb, acc_1h:0.4024282395839691, acc_4h:0.22075210511684418
AUDJPYb, acc_1h:0.2411397099494934, acc_4h:0.07967159897089005
AUDNZDb, acc_1h:0.06371772289276123, acc_4h:0.512144148349762
AUDUSDb, acc_1h:0.2926620841026306, acc_4h:0.15823344886302948
CADCHFb, acc_1h:0.5994170308113098, acc_4h:0.5889478921890259
CADJPYb, acc_1h:0.7499107122421265, acc_4h:0.1250257045030594
CHFJPYb, acc_1h:0.35174331068992615, acc_4h:0.3035100996494293
EURAUDb, acc_1h:0.36046844720840454, acc_4h:0.4345024824142456
EURCADb, acc_1h:0.22904358804225922, acc_4h:0.33468419313430786
EURCHFb, acc_1h:0.48263785243034363, acc_4h:0.639432430267334
EURGBPb, acc_1h:0.3017389178276062, acc_4h:0.450441837310791
EURJPYb, acc_1h:0.2312658727169037, acc_4h:0.37985801696777344
EURNZDb, acc_1h:0.29119136929512024, acc_4h:0.39713841676712036
EURUSDb, acc_1h:0.35006004571914673, acc_4h:0.4462292194366455
GBPAUDb, acc_1h:0.2499067634344101, acc_4h:0.31549838185310364
GBPCADb, acc_1h:0.14142611622810364, acc_4h:0.3546428978443

KeyboardInterrupt: 